In [ ]:
#principale missions : detect image 
#undestand gesture 
#provide ways to correct it 
# mission 1 classify gesture from the four principale exercices ---> add data 
# Squat , Push Up , Planck,Burpees

# Part One : Workout Exercice classification 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
types = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\train')
print (types)  

### Data preparation

In [ ]:
items = []
for item in types:
 # Get all the file names
 sub_files = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\train' + '\\' +item)
 # Add them to the list
 for exe in sub_files:
    items.append((item, str('C:\\Users\\km_ka\\Desktop\\workout\\train' + '\\' +item) + '\\' + exe))
train_df = pd.DataFrame(data=items, columns=['exercice_type', 'video_name'])
print(train_df.head())
print(train_df.tail())
df = train_df.loc[:,['video_name','exercice_type']]
df.to_csv('train.csv')

In [ ]:
from tensorflow_docs.vis import embed
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
train_df = pd.read_csv("train.csv")
print(f"Total videos for training: {len(train_df)}")
train_df.sample(10)

In [ ]:
import cv2
IMG_SIZE = 224
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
            if len(frames) == max_frames:
             break
    finally:
        cap.release()
    return np.array(frames)


In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

feature_extractor = build_feature_extractor()

In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["exercice_type"]))
print(label_processor.get_vocabulary())
labels = train_df["exercice_type"].values
labels = label_processor(labels[..., None]).numpy()
labels
print(train_df.head())


In [ ]:
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    labels = df["exercice_type"].values
    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    for idx, path in enumerate(video_paths):
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"train_labels in train set: {train_labels.shape}")





In [ ]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )
    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()
   

In [ ]:
dataset_path = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\test')
print(dataset_path)

room_types = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('C:\\Users\\km_ka\\Desktop\\workout\\test' + '\\' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('C:\\Users\\km_ka\\Desktop\\workout\\test' + '\\' +item) + '\\' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['exercice_type', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','exercice_type']]
df
df.to_csv('test.csv')

In [ ]:
test_df = pd.read_csv("test.csv")
print(f"Total videos for testing: {len(test_df)}")


In [ ]:
test_data, test_labels = prepare_all_videos(test_df, "test")
print(f"test_labels in train set: {test_labels.shape}")

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = r'C:\Users\km_ka\Desktop\workout\test\squat\pexels-mart-production-8837221 (2160p).mp4'
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


In [ ]:
from IPython.display import HTML
HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src='C:/Users/km_ka/Desktop/workout/test/squat/pexels-mart-production-8837221 (2160p).mp4' type="video/mp4" style="height:300px;width:300px">
    </video>
""")

# Part two : tracking poses


### VIDEO FEED

In [ ]:
import cv2
import mediapipe as mp 
import numpy as np

In [ ]:
drawing=mp.solutions.drawing_utils
pose=mp.solutions.pose

In [ ]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read();
    cv2.imshow('Mediapipe Feed', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

### Make detection

In [ ]:
def calculate_angle (a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    radian=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0]);
    angle=np.abs(radian*180.0/np.pi);
    if angle>180.0:
        angle=360-angle;
    return(angle)

    

In [ ]:
cap=cv2.VideoCapture(0)
with pose.Pose(min_detection_confidence = 0.5,min_tracking_confidence=0.5) as poset:
  while cap.isOpened():
    ret, frame = cap.read();
    #recoloring image 
    image=cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
   
    #Make detection 
    results = poset.process(image)
    #Recolor back to BRG

    try: 
        landmarks=results.pose_landmarks.landmark
        shoulders=[landmarks[pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow=[landmarks[pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist=[landmarks[pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[pose.PoseLandmark.LEFT_WRIST.value].y]
        arm_angle=calculate_angle(shoulders,elbow,wrist)
        print(arm_angle);
        cv2.putText(image, str(arm_angle), tuple(np.multiply(elbow, [480, 640]).astype(int)),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    except:
        pass
    #detection rendering 
    drawing.draw_landmarks(image, results.pose_landmarks , pose.POSE_CONNECTIONS)
    cv2.imshow('Mediapipe Feed', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
len(landmarks)

In [ ]:
for landmrk in pose.PoseLandmark:
    print(landmrk)